load library

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from itertools import combinations , product
import json
from functions import get_kandid , k_candidate_list , add_class_to_tuple , confidence_for_K_candidate_list  ,add_class_to_tuple , k_1_common ,remove_class , labeling , calsiffication

prepaire data set

In [ ]:
dataset = pd.read_csv('D:/AmirKabir/term 4/big data/hw1/BigData-HW1/P2/creditcard.csv')
dataset = dataset.drop(['Time'], axis=1)

#normalization v1-v28 between -1 and 1 with for loops
for i in range(1,29):
    dataset['V'+str(i)] = (dataset['V'+str(i)] - dataset['V'+str(i)].min()) / (dataset['V'+str(i)].max() - dataset['V'+str(i)].min()) * 2 - 1

#normalization amount between -1 and 1 
dataset['Amount'] = (dataset['Amount'] - dataset['Amount'].min()) / (dataset['Amount'].max() - dataset['Amount'].min()) * 2 - 1
amount_quartiles = pd.qcut(dataset['Amount'], 20 ,labels=['1', '2', '3', '4', '5', '6','7','8','9', '10', '11', '12', '13', '14', '15','16','17','18', '19', '20'])
amount_quartile_dummies = pd.get_dummies(amount_quartiles, prefix='Amount')

# Concatenate the original dataset with the new columns
dataset = pd.concat([dataset, amount_quartile_dummies], axis=1)
dataset = dataset.drop(['Amount'], axis=1)

for i in range(1,29):
    v_quartiles = pd.qcut(dataset['V'+str(i)], 20 , labels=['1', '2', '3', '4', '5', '6','7','8','9', '10', '11', '12', '13', '14', '15','16','17','18', '19', '20'])
    v_quartile_dummies = pd.get_dummies(v_quartiles, prefix='V'+str(i))
    dataset = pd.concat([dataset, v_quartile_dummies], axis=1)
    dataset = dataset.drop(['V'+str(i)], axis=1)

dataset['Class'] = dataset['Class'].replace(0, 'normal')
dataset['Class'] = dataset['Class'].replace(1, 'fraud')
class_dummies = pd.get_dummies(dataset['Class'], prefix='class')
dataset = pd.concat([dataset, class_dummies], axis=1)
dataset = dataset.drop(['Class'], axis=1)

In [ ]:
two_kandid = get_kandid(dataset)
two_kandid

In [ ]:
item_frequency = {item: dataset[item].sum() for item in dataset.columns}
item_frequency

In [ ]:
dataset_columns_list = list(dataset.columns)
dataset_columns_list.remove('class_normal')
dataset_columns_list.remove('class_fraud')
dataset_columns_list

In [ ]:
candidate_list = k_candidate_list(2,dataset_columns_list)
len(candidate_list)

In [ ]:
dic = {}
for pair in two_kandid:
    sup = len(dataset[dataset[pair[0]] + dataset[pair[1]] == 2])
    dic[pair] = sup

In [ ]:
# Define your dictionary with tuple keys
my_dict = dic

# Convert tuple keys to strings
my_dict_str = {str(k): v for k, v in my_dict.items()}

# Save dictionary to a JSON file
with open('D:/AmirKabir/term 4/big data/hw1/BigData-HW1/P2/dic.json', 'w') as fp:
    json.dump(my_dict_str, fp)

In [ ]:
# Read the dictionary from the JSON file
with open('D:/AmirKabir/term 4/big data/hw1/BigData-HW1/P2/dic.json', 'r') as fp:
    my_dict_str = json.load(fp)
# Convert string keys back to tuples
dic = {eval(k): v for k, v in my_dict_str.items()}
# Print the dictionary
print(dic)

In [ ]:
filtered_sup_two = {key: value for key, value in dic.items() if 500 < value < 800}
len(filtered_sup_two) , filtered_sup_two

In [ ]:
candidate_fraud = add_class_to_tuple(filtered_sup_two)
candidate_fraud

In [ ]:
# you can read from support_dic and confidence_dic in the zip file instead of running (for save time)
confidence_dic , support_dic = confidence_for_K_candidate_list(candidate_fraud, dataset, filtered_sup = False)
my_confidence_dic = confidence_dic
my_dict_str = {str(k): v for k, v in my_confidence_dic.items()}
with open('D:/AmirKabir/term 4/big data/hw1/BigData-HW1/P2/confidence_dic.json', 'w') as fp:
    json.dump(my_dict_str, fp)

In [ ]:
# Read the dictionary from the JSON file
with open('D:/AmirKabir/term 4/big data/hw1/BigData-HW1/P2/support_dic.json', 'r') as fp:
    my_dict_str = json.load(fp)
support_dic = {eval(k): v for k, v in my_dict_str.items()}
support_dic

In [ ]:
# Read the dictionary from the JSON file
with open('D:/AmirKabir/term 4/big data/hw1/BigData-HW1/P2/confidence_dic.json', 'r') as fp:
    my_dict_str = json.load(fp)
confidence_dic = {eval(k): v for k, v in my_dict_str.items()}
confidence_dic

In [ ]:
# calculate interest 
class_fraud_prob = len(dataset[dataset['class_fraud'] == 1]) / len(dataset)
class_normal_prob = 1 - class_fraud_prob

In [ ]:
list_confidence_dic = list(sorted(confidence_dic.values()))
list_sup_dic = list(sorted(support_dic.values()))
len(list_confidence_dic)
list_confidence_dic[-5:]

In [ ]:
# calculate interest
interest_dic_fraud_two = {}
for key in confidence_dic.keys():
    interest_dic_fraud_two[key] = confidence_dic[key] - class_fraud_prob

sorted_interest_dic_fraud_two = sorted(interest_dic_fraud_two.items(), key=lambda x: x[1], reverse=True)
sorted_interest_dic_fraud_two[:5]

In [ ]:
top_items_for_fraud = {key: value for key, value in confidence_dic.items() if value > 0.5}
top_items_for_fraud

In [ ]:
top_two_items_for_fraud = list(top_items_for_fraud.keys())
top_two_items_for_fraud

## class normal three top basket

In [ ]:
n = int(len(dic)*0.01)  # calculate 1% of the length of the dictionary
sorted_sup_two_normal = dict(sorted(dic.items(), key=lambda item: item[1], reverse=True)[:n])  
len(sorted_sup_two_normal)

In [ ]:
candidate_noraml = add_class_to_tuple(sorted_sup_two_normal)
candidate_noraml

In [ ]:
def confidence_for_K_candidate_list(K_candidate_list, dataset, filtered_sup = True ):
    confidence_dic = {}
    support_dic = {}
    for pairs in K_candidate_list: 
        objects = np.array(pairs[:-1]).reshape(1,-1)[0]
        clas =  pairs[-1]
        class_object_support = len(dataset[dataset[objects].sum(axis=1) + dataset[clas] == len(objects) + 1])
        
        if filtered_sup == True:
            filtered_sup = sorted_sup_two_normal
            object_support = filtered_sup[tuple(objects)]
        else:
            object_support = len(dataset[dataset[objects].sum(axis=1) == len(objects)])

        if object_support == 0:
            confidence = 0
            class_object_support = 0
        else:
            confidence = class_object_support / object_support

        confidence_dic[pairs] =  confidence
        support_dic[pairs] = class_object_support
    return confidence_dic , support_dic

confidence_dic_normal_three , support_dic_normal_three = confidence_for_K_candidate_list(candidate_noraml, dataset , filtered_sup= True )

In [ ]:
filt_confidenc_dic_normal_three = {key:value for key,value in sorted(confidence_dic_normal_three.items(), key=lambda item: item[1], reverse=True)}
list(filt_confidenc_dic_normal_three.items())[0:5]

In [ ]:
cal_int = list(filt_confidenc_dic_normal_three.items())[0:5]
cal_int

In [ ]:
interest_dic_fraud_two = {}
for tup in cal_int:
    interest_dic_fraud_two[tup[0]] = tup[1] - class_normal_prob
interest_dic_fraud_two

In [ ]:
final_basket_3 = {key : value for key, value in confidence_dic_normal_three.items() if value == 1 }
basket = list(final_basket_3.keys())
len(basket) , basket

## four size basket for class fraud

top_items_for_fraud = {key: value for key, value in confidence_dic.items() if value > 0.2}
top_items_for_fraud

In [ ]:
top_items_for_fraud_list = list(top_items_for_fraud.keys())
len(top_items_for_fraud_list) , top_items_for_fraud_list

In [ ]:
four_data = []
for data in top_items_for_fraud_list:
    id = data.index('class_fraud')
    data = list(data)
    data.pop(id)
    data = tuple(data)
    four_data.append(data)
four_data

In [ ]:
three_candidate = k_1_common(four_data)
len(three_candidate) , three_candidate

In [ ]:
four_candidate_list_fraud = []
three_candidate  = list(three_candidate)
for i in three_candidate:
    i = list(i)
    i.append('class_fraud')
    i = tuple(i)
    four_candidate_list_fraud.append(i)
four_candidate_list_fraud

In [ ]:
confidence_dic_four_fraud , support_dic_four_fraud = confidence_for_K_candidate_list(four_candidate_list_fraud,
 dataset , filtered_sup = False )
sorted_confidence_dic_four_fraud = dict(sorted(confidence_dic_four_fraud.items(), key=lambda item: item[1], reverse=True))
sorted_confidence_dic_four_fraud

In [ ]:
list(sorted_confidence_dic_four_fraud.items())[:5]
dic_interste_fraud = {}
cal_interset_fraud = list(sorted_confidence_dic_four_fraud.items())[:5]
for data in cal_interset_fraud:
    dic_interste_fraud[data[0]] = data[1] - class_fraud_prob
dic_interste_fraud

## four size basket for class normal

In [ ]:
final_basket_3 = {key : value for key, value in confidence_dic_normal_three.items() if value == 1 }
basket = list(final_basket_3.keys())
len(basket) ,basket

In [ ]:
top_items_for_normal_list  = basket
four_data_normal = []
for data in top_items_for_normal_list:
    id = data.index('class_normal')
    data = list(data)
    data.pop(id)
    data = tuple(data)
    four_data_normal.append(data)
len(four_data_normal) ,four_data_normal

In [ ]:
three_candidate_normal = k_1_common(four_data_normal)
three_candidate_normal ,len(three_candidate_normal)

In [ ]:
four_candidate_list_normal = []
three_candidate_normal  = list(three_candidate_normal)
for i in three_candidate_normal:
    i = list(i)
    i.append('class_normal')
    i = tuple(i)
    four_candidate_list_normal.append(i)
four_candidate_list_normal

In [ ]:
confidence_dic_four_normal , support_dic_four_normal = confidence_for_K_candidate_list(four_candidate_list_normal,
 dataset , filtered_sup= False )
sorted_confidence_dic_four_normal = dict(sorted(confidence_dic_four_normal.items(), key=lambda item: item[1], reverse=True))
len(sorted_confidence_dic_four_normal) , sorted_confidence_dic_four_normal.values()

In [ ]:
sorted_support_dic_four_normal = dict(sorted(support_dic_four_normal.items(), key=lambda item: item[1], reverse=True))
sorted_support_dic_four_normal

In [ ]:
top_sorted_confidence_dic_four_normal = {key : value for key, value in sorted_confidence_dic_four_normal.items() if value ==1}
top_sorted_confidence_dic_four_normal = list(top_sorted_confidence_dic_four_normal.keys())
len(top_sorted_confidence_dic_four_normal) , top_sorted_confidence_dic_four_normal

In [ ]:
cal_top_sorted_confidence_dic_four_normal =top_sorted_confidence_dic_four_normal[:5]
cal_top_sorted_confidence_dic_four_normal

In [ ]:
interest_dic_normal = {}
for tup in cal_top_sorted_confidence_dic_four_normal:
    interest_dic_normal[tup[0]] = tup[1] - class_normal_prob
interest_dic_normal  

## classification task

top_sorted_confidence_dic_four_normal = remove_class(top_sorted_confidence_dic_four_normal)
top_sorted_confidence_dic_four_fraud = remove_class(list(sorted_confidence_dic_four_fraud.keys())[:5])
final_basket_3 = remove_class(list(final_basket_3.keys()))
top_two_items_for_fraud = remove_class(top_two_items_for_fraud)

In [ ]:
#list of all the rules and labels
normal_label = 'normal'
fraud_label = 'fraud'
A = ([tuple(sorted(tup)) for tup in top_sorted_confidence_dic_four_normal] ,  normal_label)
B = ([tuple(sorted(tup)) for tup in top_sorted_confidence_dic_four_fraud] , fraud_label)
C = ([tuple(sorted(tup)) for tup in final_basket_3] , normal_label)
D = ([tuple(sorted(tup)) for tup in top_two_items_for_fraud] , fraud_label)
final_list_fraud = [B,D]

In [ ]:
predicted_label_list = labeling(dataset)
count = 0
for i in predicted_label_list:
    if i == 1:
        count += 1
count

In [ ]:
accuracy , precision , recall , f1_score , TP ,  FP , TN , FN = calsiffication(predicted_label_list , first_dataset)
accuracy , precision , recall , f1_score
TP ,  FP , TN , FN